In [ ]:
# default_exp engine.gmm_loss

In [ ]:
%load_ext autoreload
%autoreload 2

# Loss function

In [ ]:
#export
from decode_fish.imports import *
from torch import distributions as D, Tensor
from torch.distributions import Distribution

In [ ]:
#export
class PointProcessGaussian(Distribution):
    def __init__(self, logits: torch.tensor, xyzi_mu: torch.tensor,
                 xyzi_sigma: torch.tensor, **kwargs):
        "logits: BS, C, H, W, D"
        self.logits = logits
        self.xyzi_mu = xyzi_mu
        self.xyzi_sigma = xyzi_sigma

    def sample(self, N:int =1):
        if N ==1:
            return self._sample()
       
        res_ = [self._sample() for i in range(N)]
        locations   =  torch.cat([i[0] for i in res_], dim=1)
        x_offset    =  torch.cat([i[1] for i in res_], dim=1)
        y_offset    =  torch.cat([i[2] for i in res_], dim=1)
        z_offset    =  torch.cat([i[3] for i in res_], dim=1)
        intensities =  torch.cat([i[4] for i in res_], dim=1)
        return locations, x_offset, y_offset, z_offset, intensities



    def _sample(self):
        #ask about this sampling
        locations = D.Bernoulli(logits=self.logits).sample()
        xyzi = D.Independent(D.Normal(self.xyzi_mu, self.xyzi_sigma),
                             1).sample()
        x_offset, y_offset, z_offset, intensities = (i.unsqueeze(1) for i in
                                  torch.unbind(xyzi, dim=1))
        
        output_shape = tuple(locations.shape)
        locations = locations.nonzero(as_tuple=True)
        x_offset  = x_offset[locations]
        y_offset  = y_offset[locations]        
        z_offset  = z_offset[locations]
        intensities = intensities[locations]
        return locations, x_offset, y_offset, z_offset, intensities, output_shape


    def log_prob(self, locations_3d, x_offset_3d, y_offset_3d, z_offset_3d, intensities_3d):
        
        batch_size = self.logits.shape[0]
        xyzi, counts, s_mask = get_true_labels(batch_size, locations_3d, x_offset_3d, y_offset_3d, z_offset_3d, intensities_3d )
        x_mu, y_mu, z_mu, i_mu = (i.unsqueeze(1) for i in
                                  torch.unbind(self.xyzi_mu, dim=1))
        x_si, y_si, z_si, i_si = (i.unsqueeze(1) for i in
                                  torch.unbind(self.xyzi_sigma, dim=1))
        
        P = torch.sigmoid(self.logits) # + 0.00001
        count_mean = P.sum(dim=[2, 3, 4]).squeeze(-1)
        count_var = (P - P ** 2).sum(dim=[2, 3, 4]).squeeze(-1)  #avoid situation where we have perfect match
        count_dist = D.Normal(count_mean, torch.sqrt(count_var))
                
        count_prob =  count_dist.log_prob(counts) # * counts
        
        mixture_probs = P / P.sum(dim=[2, 3, 4], keepdim=True)
        
        xyz_mu_list, _, _, i_mu_list, x_sigma_list, y_sigma_list, z_sigma_list, i_sigma_list, mixture_probs_l = img_to_coord(
            batch_size, P, x_mu, y_mu, z_mu, i_mu, x_si, y_si, z_si, i_si, mixture_probs)
        xyzi_mu = torch.cat((xyz_mu_list, i_mu_list), dim=-1)
        xyzi_sigma = torch.cat((x_sigma_list, y_sigma_list, z_sigma_list, i_sigma_list), dim=-1) #to avoind NAN
        mix = D.Categorical(mixture_probs_l.squeeze(-1))
        comp = D.Independent(D.Normal(xyzi_mu, xyzi_sigma + 0.00001), 1)
        spatial_gmm = D.MixtureSameFamily(mix, comp)
        spatial_prob = spatial_gmm.log_prob(xyzi.transpose(0, 1)).transpose(0,1)
        spatial_prob = (spatial_prob * s_mask).sum(-1)
        count_prob, spatial_prob
        return count_prob, spatial_prob
    
def img_to_coord(bs, locations, x_os, y_os, z_os, *args):
    """
    Given `locations'  will extract value of x_os, y_os, z_os where probability is more than 0.
    also generates counts of location and returns mask 
    
    Args:
        locations: Tuple(BS, Frames, D, H, W)
        x_offsets: (N_emitters,)
        y_offsets: (N_emitters,)
        z_offsets: (N_emitters,)
        intensities: (N_emitters,)

    """

    if type(locations) == torch.Tensor:
        #this is done for model outputs since they are volumes
        locations = torch.nonzero(locations,as_tuple=True)
        x_os      = x_os[locations]
        y_os      = y_os[locations]
        z_os      = z_os[locations] 
        a = [item[locations].unsqueeze(-1) for item in args]
    else:    
        a = [item.unsqueeze(-1) for item in args]
        
    # D == z 2, H == y 3, W == x 4
    x =  x_os + locations[4].type(torch.cuda.FloatTensor) + 0.5 
    y =  y_os + locations[3].type(torch.cuda.FloatTensor) + 0.5 
    z =  z_os + locations[2].type(torch.cuda.FloatTensor) + 0.5 

    xyz =  torch.stack((x, y, z), dim=1)
    
    #to match where in batch we have a counts (if there is no count at this 
    #position we will get 0 in tensor
    counts_ = torch.unique(locations[0], return_counts=True)[1]
    batch_loc = torch.unique(locations[0])
    
    counts = torch.cuda.LongTensor(bs).fill_(0)
    counts[batch_loc] = counts_
    
    max_counts = counts.max()
    if max_counts==0: max_counts = 1 #if all 0 will return empty matrix of correct size
    xyz_list = torch.cuda.FloatTensor(bs,max_counts,3).fill_(0)
    i_list   = [torch.cuda.FloatTensor(bs,max_counts,1).fill_(0) for i in range(len(a))]
#     s_arr    = torch.cat([torch.arange(c) for c in counts], dim = 0)
    s_arr = cum_count_per_group(locations[0])
    s_mask   = torch.cuda.FloatTensor(bs,max_counts).fill_(0)
    s_mask[locations[0],s_arr] = 1
    xyz_list[locations[0],s_arr] = xyz
    for i,k in zip(i_list, a): i[locations[0],s_arr] = k
    return (xyz_list, counts, s_mask) + tuple(i_list)
    
def get_true_labels(bs, locations, x_os, y_os, z_os, ints):
    xyz_list, counts_true, s_mask, i_1 = img_to_coord(bs, locations, x_os, y_os, z_os, ints)
    xyzi_true = torch.cat((xyz_list, i_1), dim=-1)
    return xyzi_true, counts_true, s_mask

def grp_range(counts: torch.Tensor):
    assert counts.dim() == 1

    idx = counts.cumsum(0)
    id_arr = torch.ones(idx[-1], dtype=int, device=counts.device)
    id_arr[0] = 0
    id_arr[idx[:-1]] = -counts[:-1] + 1
    return id_arr.cumsum(0)

def cum_count_per_group(arr):
    """
    Helper function that returns the cumulative sum per group.
    Example:
        [0, 0, 0, 1, 2, 2, 0] --> [0, 1, 2, 0, 0, 1, 3]
    """

    if arr.numel() == 0:
        return arr

    _, cnt = torch.unique(arr, return_counts=True)
    return grp_range(cnt)[np.argsort(np.argsort(arr.cpu().numpy(), kind='mergesort'), kind='mergesort')]

In [ ]:
model_out = torch.load('../data/model_batch_output.pt')
model_out.keys()
model_out['logits'].shape

/groups/turaga/home/speisera/anaconda3/envs/decode2_dev/lib/python3.7/site-packages/torch/cuda/__init__.py:81: UserWarning: 
    Found GPU2 NVS 510 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


torch.Size([2, 1, 48, 48, 48])

In [ ]:
from decode_fish.engine.point_process import PointProcessUniform
point_process = PointProcessUniform(local_rate = torch.ones([2,1,48,48,48])*.0001, min_int = 0.5, sim_iters=5)
locs_3d, x_os_3d, y_os_3d, z_os_3d, ints_3d, output_shape = point_process.sample()

In [ ]:
locs_3d = [l.cuda() for l in locs_3d]
xyzi_true, counts_true, s_mask = get_true_labels(2, locs_3d, x_os_3d.cuda(), y_os_3d.cuda(), z_os_3d.cuda(), ints_3d.cuda())
print(len(locs_3d[0]))
print(counts_true)
print(s_mask)
print(s_mask.sum(-1))

19
tensor([12,  7], device='cuda:0')
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.]], device='cuda:0')
tensor([12.,  7.], device='cuda:0')


In [ ]:
model_out['logits'][0].shape
print(torch.sigmoid(model_out['logits'][0]).sum())
print(torch.sigmoid(model_out['logits'][1]).sum())

tensor(6., device='cuda:0', grad_fn=<SumBackward0>)
tensor(10.4737, device='cuda:0', grad_fn=<SumBackward0>)


In [ ]:
model_out['logits'][0] = model_out['logits'][0]*0 - 50
model_out['logits'][0,0,0,0,:11] = 50

In [ ]:
gmm_loss = PointProcessGaussian(**model_out).log_prob(locs_3d, x_os_3d.cuda(), y_os_3d.cuda(), z_os_3d.cuda(), ints_3d.cuda())

tensor([11.0000, 10.4737], device='cuda:0', grad_fn=<SqueezeBackward1>) tensor([2.1328e-17, 8.1571e+00], device='cuda:0', grad_fn=<SqueezeBackward1>) tensor([12,  7], device='cuda:0')


In [ ]:
gmm_loss

(tensor([-2.8132e+17, -1.8956e+01], device='cuda:0', grad_fn=<MulBackward0>),
 tensor([-665.1628, -265.1302], device='cuda:0', grad_fn=<SumBackward1>))

In [ ]:
from decode_fish.funcs.utils import free_mem
free_mem()

In [ ]:
!nbdev_build_lib

Converted 00_models.ipynb.
Converted 01_psf.ipynb.
Converted 02_microscope.ipynb.
Converted 03_noise.ipynb.
Converted 04_pointsource.ipynb.
Converted 05_gmm_loss.ipynb.
Converted 06_plotting.ipynb.
Converted 07_file_io.ipynb.
Converted 08_dataset.ipynb.
Converted 09_output_trafo.ipynb.
Converted 10_evaluation.ipynb.
Converted 11_emitter_io.ipynb.
Converted 12_utils.ipynb.
Converted 13_train.ipynb.
Converted 15_fit_psf.ipynb.
Converted 16_visualization.ipynb.
Converted index.ipynb.
